In [1]:
# Data Management/Investigation
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import numpy as np
import missingno as miss
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import sqlite3 as sql

Contents of this script:

-process the demographic data

-merge that data into the larger data frame

-export the final cleaned data as a new, separate .csv

### Cleaning the Demographic Data!

Notes:

-This data set is the largest of the three I am using, and it's not even close. Unaltered, it has almost three quarters of a million rows, and processing it takes time. For this project, I am investigating factors which predict a democratic or republican win for a county in the 2016 presidential election. To make this data set workable, I am going to drop all the non-2016 data (as much as I'd love to see the effects of different demographic makeups over time, I just don't have the processing power to make that happen right now).

-I am also going to drop the data for several age categories, specifically the groups where some or all of the included ages are less than 18. Minors cannot vote, and since my overall question pertains to voting, I do not think it makes sense to keep those rows in the data frame. (Plus, anything which helps trim this behemoth down seems like a good plan to me.)

-Additionally, I am dropping a bunch of columns for different racial categories. The codebook for this data set ("Race_Sex_Age_2010-2019_Codebook.pdf" in the folder "Codebooks") lists each of these columns. For the most part, they reference different combinations of races (i.e. the number of people in an age group-county-year who are white AND hispanic, rather than just white or just hispanic).

-Finally, I am not going to incorporate age as a variable in this analysis. Before I added this part to this initial notes section, I tried multiple times to write functions which would rearrange the data based on age category, and each time the function would run for upwards of 10 to 15 minutes and still not be finished. As much as I would love to use age in this analysis, I do not have the ability to properly process it for this project. 

In [97]:
#Connect to the database
conn = sql.connect('aja149_ppol564__finalproject_sql_database.db')

In [98]:
pop_data = pd.read_sql_query("""SELECT *
                                FROM demographics;""",conn)

In [99]:
#Take the data from the first row and use it for the column names

new_header = pop_data.iloc[0] #grab the first row for the header
pop_data = pop_data[1:] #take the data less the header row
pop_data.columns = new_header #set the header row as the df header

In [103]:
pop_data.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'H_MALE', 'H_FEMALE', 'FIPS'],
      dtype='object', name=0)

In [101]:
#These three lines will drop all the unneeded racial categories (there are almost 60)
pop_data=pop_data.drop(pop_data.filter(like='C_',axis=1).columns,axis=1)
pop_data=pop_data.drop(pop_data.filter(like='NH',axis=1).columns,axis=1)
pop_data=pop_data.drop(columns = ['HWA_MALE','HWA_FEMALE','HBA_MALE','HBA_FEMALE','HIA_MALE','HIA_FEMALE',
                                   'HAA_MALE','HAA_FEMALE','HNA_MALE','HNA_FEMALE','HTOM_MALE','HTOM_FEMALE'])

In [102]:
#Also, create a new column containing the FIPS for each county, to provide a common unit to make it easier to merge this
#data with the other data
pop_data['FIPS'] = pop_data['STATE']+pop_data['COUNTY']

In [104]:
#Keep only the rows where the codebook specifies the year is 2016
pop_data = pop_data[pop_data.YEAR.isin(['9'])]

#Drop the rows where age is less than 18 (including the group which spans 15-19 year olds)
pop_data = pop_data[pop_data.AGEGRP.isin(['0'])]

In [105]:
#Convert the number data to numeric
cols_num = pop_data.drop(columns = ['SUMLEV', 'STATE', 'COUNTY','STNAME','CTYNAME','YEAR','AGEGRP','FIPS'])
cols_num = cols_num.apply(lambda x: x.str.replace(',', '').astype(float), axis=1)
cols_num = cols_num.apply(pd.to_numeric)

#Create a second slice with just the string columns
cols_str = pop_data[['SUMLEV', 'STATE', 'COUNTY','STNAME','CTYNAME','FIPS']]
#Concatenate the two slices, first string, then num
pop_data = pd.concat([cols_str, cols_num], axis = 1)

In [110]:
#Ok, the next step is to convert each demographic column to proportions instead of integers. 
def proportion():
    col_names = ['TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'H_MALE', 'H_FEMALE']
    
    for ii in col_names:
        pop_data[ii] = pop_data[ii]/pop_data['TOT_POP']
        
    return(pop_data)

In [111]:
pop_data = proportion()

In [115]:
pop_data = pop_data.drop(columns = ['SUMLEV','STATE','COUNTY','STNAME','CTYNAME'])
pop_data.head()

,FIPS,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,H_MALE,H_FEMALE
153,01001,55243.0,0.488207,0.511793,0.379759,0.392810,0.091233,0.100682,0.002154,0.002516,0.005195,0.006499,0.000525,0.000398,0.009341,0.008888,0.014174,0.012943
381,01003,207601.0,0.486361,0.513639,0.425350,0.447170,0.044451,0.047519,0.003979,0.003661,0.003979,0.006214,0.000299,0.000284,0.008304,0.008791,0.023526,0.021228
609,01005,25806.0,0.528017,0.471983,0.262691,0.235255,0.252461,0.227001,0.003526,0.002325,0.002093,0.002248,0.001085,0.000504,0.006161,0.004650,0.024103,0.019027
837,01007,22586.0,0.534402,0.465598,0.391349,0.375631,0.134464,0.082042,0.001904,0.001815,0.000841,0.001063,0.001018,0.000177,0.004826,0.004870,0.014080,0.010626
1065,01009,57494.0,0.491999,0.508001,0.472745,0.487947,0.008349,0.007427,0.002992,0.003009,0.001235,0.001513,0.000678,0.000417,0.006001,0.007688,0.048179,0.042404


In [128]:
final_df = pd.read_csv('final_df.csv')

In [129]:
final_df.columns

Index(['FIPS', 'Stabr', 'area_name', 'Rural_urban_continuum_code_2013',
       'Urban_influence_code_2013', 'Metro_2013',
       ' Civilian_labor_force_2000 ', ' Employed_2000 ', ' Unemployed_2000 ',
       'Unemployment_rate_2000', ' Civilian_labor_force_2001 ',
       ' Employed_2001 ', ' Unemployed_2001 ', 'Unemployment_rate_2001',
       ' Civilian_labor_force_2002 ', ' Employed_2002 ', ' Unemployed_2002 ',
       'Unemployment_rate_2002', ' Civilian_labor_force_2003 ',
       ' Employed_2003 ', ' Unemployed_2003 ', 'Unemployment_rate_2003',
       ' Civilian_labor_force_2004 ', ' Employed_2004 ', ' Unemployed_2004 ',
       'Unemployment_rate_2004', ' Civilian_labor_force_2005 ',
       ' Employed_2005 ', ' Unemployed_2005 ', 'Unemployment_rate_2005',
       ' Civilian_labor_force_2006 ', ' Employed_2006 ', ' Unemployed_2006 ',
       'Unemployment_rate_2006', ' Civilian_labor_force_2007 ',
       ' Employed_2007 ', ' Unemployed_2007 ', 'Unemployment_rate_2007',
       ' Civilian_

In [130]:
#Drop some columns I should have dropped in the earlier processing stage since I only care about unemployment rate and not
#labor force size
final_df = final_df.drop(columns = [' Civilian_labor_force_2000 ', ' Employed_2000 ', ' Unemployed_2000 ',
       ' Civilian_labor_force_2001 ',' Employed_2001 ', ' Unemployed_2001 ',' Civilian_labor_force_2002 ', 
       ' Employed_2002 ', ' Unemployed_2002 ',' Civilian_labor_force_2003 ',' Employed_2003 ', ' Unemployed_2003 ',
       ' Civilian_labor_force_2004 ', ' Employed_2004 ', ' Unemployed_2004 ',' Civilian_labor_force_2005 ',
       ' Employed_2005 ', ' Unemployed_2005 ',' Civilian_labor_force_2006 ', ' Employed_2006 ', ' Unemployed_2006 ',
       ' Civilian_labor_force_2007 ',' Employed_2007 ', ' Unemployed_2007 ',' Civilian_labor_force_2008 ', 
       ' Employed_2008 ', ' Unemployed_2008 ','Civilian_labor_force_2009', 'Employed_2009','Unemployed_2009',
       ' Civilian_labor_force_2010 ', ' Employed_2010 ', ' Unemployed_2010 ',' Civilian_labor_force_2011 ',
       ' Employed_2011 ', ' Unemployed_2011 ',' Civilian_labor_force_2012 ', ' Employed_2012 ', ' Unemployed_2012 ',
       ' Civilian_labor_force_2013 ',' Employed_2013 ', ' Unemployed_2013 ',' Civilian_labor_force_2014 ', 
       ' Employed_2014 ', ' Unemployed_2014 ',' Civilian_labor_force_2015 ',' Employed_2015 ', ' Unemployed_2015 ',
       ' Civilian_labor_force_2016 ', ' Employed_2016 ', ' Unemployed_2016 ', 'state', 'state_po', 'County_equivalent'])

In [131]:
#Convert FIPS to string to make merge possible
final_df['FIPS'] = final_df['FIPS'].astype(str)
#Fill in leading zeroes so the merges happen on the correct values
final_df['FIPS'] = final_df['FIPS'].str.zfill(5)
#Merge the population data in with the other data and show the first 5 rows just to check
final_df = pd.merge(final_df,pop_data,how = 'outer',on='FIPS')
final_df.head()

,FIPS,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Unemployment_rate_2000,Unemployment_rate_2001,Unemployment_rate_2002,Unemployment_rate_2003,Unemployment_rate_2004,Unemployment_rate_2005,Unemployment_rate_2006,Unemployment_rate_2007,Unemployment_rate_2008,Unemployment_rate_2009,Unemployment_rate_2010,Unemployment_rate_2011,Unemployment_rate_2012,Unemployment_rate_2013,Unemployment_rate_2014,Unemployment_rate_2015,Unemployment_rate_2016,Dvotes2000,Rvotes2000,tvotes2000,Dvotes2004,Rvotes2004,tvotes2004,Dvotes2008,Rvotes2008,tvotes2008,Dvotes2012,Rvotes2012,tvotes2012,Dvotes2016,Rvotes2016,tvotes2016,2P_2000,2P_2004,2P_2008,2P_2012,D_win2016,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,H_MALE,H_FEMALE
0,01001,AL,"Autauga County, AL",2.0,2.0,1.0,4.0,4.1,4.8,5.1,4.8,3.8,3.3,3.3,5.1,9.7,8.9,8.4,6.9,6.2,5.8,5.0,5.1,0.291822,0.708178,17208.0,0.238448,0.761552,20081.0,0.259321,0.740679,23641.0,0.268006,0.731994,23932.0,5936.0,18172.0,24973.0,16935.0,19954.0,23496.0,23742.0,0.0,55243.0,0.488207,0.511793,0.379759,0.392810,0.091233,0.100682,0.002154,0.002516,0.005195,0.006499,0.000525,0.000398,0.009341,0.008888,0.014174,0.012943
1,01003,AL,"Baldwin County, AL",3.0,2.0,1.0,3.7,4.3,5.0,5.0,5.2,4.0,3.2,3.1,4.6,9.8,10.0,9.0,7.5,6.6,6.1,6.0,5.3,0.255099,0.744901,56480.0,0.227490,0.772510,69320.0,0.240351,0.759649,81413.0,0.218190,0.781810,85338.0,18458.0,72883.0,95215.0,54869.0,68570.0,80657.0,84440.0,0.0,207601.0,0.486361,0.513639,0.425350,0.447170,0.044451,0.047519,0.003979,0.003661,0.003979,0.006214,0.000299,0.000284,0.008304,0.008791,0.023526,0.021228
2,01005,AL,"Barbour County, AL",6.0,6.0,0.0,5.5,7.4,7.7,7.1,7.2,5.8,5.7,6.3,8.8,14.3,12.3,11.5,11.5,10.2,10.5,9.0,8.3,0.504473,0.495527,10395.0,0.450284,0.549716,10777.0,0.492692,0.507308,11630.0,0.515791,0.484209,11509.0,4871.0,5454.0,10469.0,10284.0,10731.0,11563.0,11462.0,0.0,25806.0,0.528017,0.471983,0.262691,0.235255,0.252461,0.227001,0.003526,0.002325,0.002093,0.002248,0.001085,0.000504,0.006161,0.004650,0.024103,0.019027
3,01007,AL,"Bibb County, AL",1.0,1.0,1.0,5.3,6.8,7.0,6.0,5.5,4.5,4.2,4.1,5.8,13.3,11.4,10.5,8.5,7.9,7.2,7.0,6.4,0.388085,0.611915,7101.0,0.276286,0.723714,7600.0,0.268543,0.731457,8644.0,0.264219,0.735781,8420.0,1874.0,6738.0,8819.0,6983.0,7561.0,8561.0,8334.0,0.0,22586.0,0.534402,0.465598,0.391349,0.375631,0.134464,0.082042,0.001904,0.001815,0.000841,0.001063,0.001018,0.000177,0.004826,0.004870,0.014080,0.010626
4,01009,AL,"Blount County, AL",1.0,1.0,1.0,3.5,3.6,5.4,4.6,4.2,3.6,3.2,3.2,4.7,10.0,9.8,8.7,6.9,6.3,6.1,5.0,5.4,0.282079,0.717921,17973.0,0.184675,0.815325,21504.0,0.147296,0.852704,24267.0,0.125174,0.874826,24006.0,2156.0,22859.0,25588.0,17644.0,21324.0,23911.0,23727.0,0.0,57494.0,0.491999,0.508001,0.472745,0.487947,0.008349,0.007427,0.002992,0.003009,0.001235,0.001513,0.000678,0.000417,0.006001,0.007688,0.048179,0.042404


In [132]:
#Check for missingness
final_df.isna().sum()

FIPS                                0
Stabr                               3
area_name                           3
Rural_urban_continuum_code_2013     3
Urban_influence_code_2013           3
Metro_2013                          3
Unemployment_rate_2000              3
Unemployment_rate_2001              3
Unemployment_rate_2002              3
Unemployment_rate_2003              3
Unemployment_rate_2004              3
Unemployment_rate_2005              3
Unemployment_rate_2006              3
Unemployment_rate_2007              3
Unemployment_rate_2008              3
Unemployment_rate_2009              3
Unemployment_rate_2010              3
Unemployment_rate_2011              3
Unemployment_rate_2012              3
Unemployment_rate_2013              3
Unemployment_rate_2014              3
Unemployment_rate_2015              3
Unemployment_rate_2016              3
Dvotes2000                         83
Rvotes2000                         83
tvotes2000                         83
Dvotes2004  

In [133]:
#Drop the rows with missing values
final_df = final_df.dropna()
#Final shape: 3139 rows, 60 columns
final_df.shape

(3139, 60)

I am now going to export this data frame as a .csv so it can be loaded into the visualization and modeling script.

In [134]:
final_df.to_csv('aja149_cleandata.csv', index=False)

In [135]:
conn.close()